In [ ]:
import numpy as np
import pandas as pd
from math import pi
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.models import Model
from keras.layers import GRU, Dense, Dropout, concatenate, Input
from keras.optimizers import RMSprop
np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
data = pd.read_csv('./historic_data.csv')
data['actual'] = data['rides']
train_cut = int(len(data) * 0.8)
validate_cut = int(len(data) * 0.9)

In [3]:
X, y = data.iloc[:, data.columns != 'rides'], data.iloc[:, data.columns == 'rides'].iloc[:, 0]
X.head()

,date,hour,month,day_of_week,sunrise,icon,precip_prob,temperature,humidity,wind_speed,actual
0,2013-06-01,0,6,7,0,clear,0.01,77.65,0.61,2.06,152
1,2013-06-01,1,6,7,0,clear,0.01,75.62,0.67,1.93,102
2,2013-06-01,2,6,7,0,clear,0.01,74.72,0.70,2.31,67
3,2013-06-01,3,6,7,0,clear,0.01,73.32,0.76,2.16,41
4,2013-06-01,4,6,7,0,clear,0.01,72.42,0.79,1.93,16


In [8]:
class YearExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.applymap(lambda x: float(pd.to_datetime(x).year))

class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.loc[:, self.columns]

class CustomTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, func):
        self.func = func
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return self.func(X)

In [9]:
pipeline = Pipeline([
    ('union', FeatureUnion([
        ('cat', Pipeline([
            ('cat_selector', ColumnSelector(['hour', 'day_of_week', 'month'])),
            ('cat_encoder', OneHotEncoder(sparse=False))
        ])),
        ('year', Pipeline([
            ('year_selector', ColumnSelector(['date'])),
            ('year_extractor', YearExtractor()),
        ])),
        ('int', Pipeline([
            ('int_selector', ColumnSelector(['sunrise', 'precip_prob', 'temperature', 'humidity', 'wind_speed', 'actual'])),
        ])),
    ])),
    ('scaler', MinMaxScaler())
])

In [10]:
pipeline.fit(X.iloc[0:train_cut, :])
ft_X = pipeline.transform(X)

C:\Users\Brian\Anaconda3\envs\citibike-dashboard\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
from sklearn.externals import joblib

joblib.dump(pipeline, 'dl_X_pipeline.pkl', compress=1)

['dl_X_pipeline.pkl']

In [12]:
scaler = MinMaxScaler()
scaler.fit(np.array(y[0:train_cut]).reshape(-1, 1))

min_max_y = scaler.transform(np.array(y).reshape(-1, 1))

C:\Users\Brian\Anaconda3\envs\citibike-dashboard\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [13]:
joblib.dump(scaler, 'dl_y_pipeline.pkl', compress=1)

['dl_y_pipeline.pkl']

In [17]:
def generator(predictors, response, start, stop, lookback, lag, batch_size=2):
    index = start
    while True:
        if index + batch_size > stop:
            samples = np.zeros((stop-index, lookback, 1))
            meta = np.zeros((stop-index, len(predictors[0])-1))
            targets = np.zeros(stop-index)
        else:
            samples = np.zeros((batch_size, lookback, 1))
            meta = np.zeros((batch_size, len(predictors[0])-1))
            targets = np.zeros(batch_size)
        for i in range(samples.shape[0]):
            samples[i] = predictors[index-lookback-lag+i:index-lag+i, -1].reshape(-1, 1)
            meta[i] = predictors[index + i, :-1]
            targets[i] = response[index + i]

        index += batch_size
        if index >= stop:
            index = start
        
        yield [samples, meta], targets

In [18]:
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=20
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10
    ),
    ModelCheckpoint(
        filepath='my_model6.h5',
        monitor='val_loss',
        save_best_only=True
    )
]
        
lookback = 7*24 # seven days
lag = 1*24 # one day
batch_size = 64
train_steps = (train_cut - lookback - lag) // batch_size + 1
val_steps = (validate_cut - train_cut) // batch_size + 1


train_gen = generator(ft_X, min_max_y, lookback+lag, train_cut, lookback, lag, batch_size)
validate_gen = generator(ft_X, min_max_y, train_cut, validate_cut, lookback, lag, batch_size)

In [19]:
historical = Input(shape=(lookback, 1), name='historical')
historical1 = GRU(64, input_shape=(None, ))(historical)
historical1 = Dense(64, activation='relu')(historical1)
historical1 = Dropout(0.5)(historical1)

weather = Input(shape=(49, ), name='weather')
weather1 = Dense(64, activation='relu')(weather)
weather1 = Dropout(0.5)(weather1)

concat = concatenate([historical1, weather1])
concat1 = Dense(128, activation='relu')(concat)
concat1 = Dropout(0.5)(concat1)
concat1 = Dense(64, activation='relu')(concat1)
concat1 = Dropout(0.5)(concat1)
output = Dense(1)(concat1)

model = Model([historical, weather], output)
model.compile(optimizer=RMSprop(), loss='mse', metrics=['mae'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
historical (InputLayer)         (None, 168, 1)       0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, 64)           12672       historical[0][0]                 
__________________________________________________________________________________________________
weather (InputLayer)            (None, 49)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           4160        gru_1[0][0]                      
__________________________________________________________________________________________________
dense_2 (D

In [36]:
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=75, 
                    validation_data=validate_gen, validation_steps=val_steps,
                    callbacks=callbacks_list)

Epoch 1/75
495/495 [==============================] - 130s 262ms/step - loss: 0.0137 - mean_absolute_error: 0.0825 - val_loss: 0.0284 - val_mean_absolute_error: 0.1200
Epoch 2/75
495/495 [==============================] - 86s 173ms/step - loss: 0.0062 - mean_absolute_error: 0.0560 - val_loss: 0.0203 - val_mean_absolute_error: 0.0986
Epoch 3/75
495/495 [==============================] - 85s 171ms/step - loss: 0.0049 - mean_absolute_error: 0.0498 - val_loss: 0.0165 - val_mean_absolute_error: 0.0899
Epoch 4/75
495/495 [==============================] - 85s 171ms/step - loss: 0.0043 - mean_absolute_error: 0.0459 - val_loss: 0.0133 - val_mean_absolute_error: 0.0797
Epoch 5/75
495/495 [==============================] - 84s 171ms/step - loss: 0.0039 - mean_absolute_error: 0.0436 - val_loss: 0.0138 - val_mean_absolute_error: 0.0794
Epoch 6/75
495/495 [==============================] - 85s 171ms/step - loss: 0.0036 - mean_absolute_error: 0.0418 - val_loss: 0.0115 - val_mean_absolute_error: 0.07

In [31]:
from sklearn.metrics import mean_squared_error

model.load_weights('my_model6.h5')
model.compile(optimizer=RMSprop(), loss='mse', metrics=['mae'])

new_gen = generator(ft_X, min_max_y, train_cut, validate_cut, lookback, lag, batch_size)
predictions = model.predict_generator(new_gen, steps=val_steps)
mean_squared_error(scaler.inverse_transform(predictions), y[train_cut:validate_cut])

210985.83112290205